In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
dataset=pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
data=dataset.copy()

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(data,data['stroke']):
  strat_train_set=data.loc[train_index]
  strat_test_set=data.loc[test_index]

In [ ]:
data=strat_train_set.copy()

In [ ]:
predictor_variables=list(data.columns)

In [ ]:
predictor_variables.remove('stroke')

In [ ]:
target_variables=['stroke']

In [ ]:
character_variables=[variable for variable in data.columns if data[variable].dtypes=='O']

In [ ]:
character_variables

In [ ]:
numeric_variables=[variable for variable in data.columns if data[variable].dtypes!='O']

In [ ]:
numeric_variables

In [ ]:
categorical_variables=[feature for feature in data.columns if len(data[feature].value_counts())<25]

In [ ]:
len(categorical_variables)

In [ ]:
continuous_variables=[feature for feature in data.columns if len(data[feature].value_counts())>25]

In [ ]:
len(continuous_variables)

In [ ]:
##Univariate analysis

In [ ]:
data[continuous_variables].describe()

In [ ]:
data[continuous_variables].hist(bins=100,figsize=(20,15))
plt.show()

In [ ]:
sns.set_theme(style="whitegrid")
for feature in continuous_variables:
  sns.boxplot(y=data[feature])
  plt.show()

In [ ]:
for feature in categorical_variables:
  print((data[feature].value_counts()/len(data)))

In [ ]:
for feature in categorical_variables:
  data[feature].value_counts().plot(kind='bar')
  plt.figure(figsize=(5,5))
  plt.show()

In [ ]:
##bivariate analysis

In [ ]:
##for variable in continuous_variables:
##    sns.boxplot(x=variable,y='stroke',data=data)
##    plt.show()

In [ ]:
for variable in categorical_variables:
  data_crosstab = pd.crosstab(data['stroke'], 
                            data[variable],
                                margins = False)
  print(data_crosstab)
  print('\n')

In [ ]:
##Handling missing values

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
df=data.copy()
cat_imp=SimpleImputer(strategy='most_frequent')
cont_imp=SimpleImputer(strategy='median')
list_cat=[v for v in categorical_variables if v!='stroke']
df[list_cat]=cat_imp.fit_transform(df[list_cat])
df[continuous_variables]=cont_imp.fit_transform(df[continuous_variables])


In [ ]:
data=df

In [ ]:
##outlier_detection

In [ ]:
## converting character encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
full_pipeline=ColumnTransformer([
                                 ('num',StandardScaler(),[variable for variable in numeric_variables if variable!='stroke']),
                                 ('cat',OneHotEncoder(),character_variables)
])

In [ ]:
data_prepared=full_pipeline.fit_transform(data)

In [ ]:
data_prepared

In [ ]:
data_prepared.shape

In [ ]:
X=data_prepared.copy()

In [ ]:
y=data['stroke']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [ ]:
score=cross_val_score(RandomForestClassifier(max_depth=2),X,y,cv=10)

In [ ]:
score.mean()

In [ ]:
score=cross_val_score(LogisticRegression(),X,y,cv=3)

In [ ]:
score.mean()

In [ ]:
score=cross_val_score(SVC(),X,y,cv=3)

In [ ]:
score.mean()

In [ ]:
rnd_clf=RandomForestClassifier(max_depth=2)

In [ ]:
rnd_clf.fit(X,y)

In [ ]:
## creating the test set

In [ ]:
X_test=strat_test_set

In [ ]:
y_test=X_test['stroke']

In [ ]:
X_test[list_cat]=cat_imp.transform(X_test[list_cat])

In [ ]:
X_test[continuous_variables]=cont_imp.transform(X_test[continuous_variables])

In [ ]:
X_test=full_pipeline.transform(X_test)

In [ ]:
y_pred=rnd_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
score=accuracy_score(y_test,y_pred)

In [ ]:
score